In [ ]:
import torch
import numpy as np
import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
batch_size = 25
learning_rate = 0.001
num_epochs = 30

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [ ]:
class CollectData(Dataset):
    def __init__(self, x):
        self.X = x
        
    def __len__(self):
        return len(self.X)
    def __getitem__(self, index):
        label = torch.tensor(self.X[index][1])
        img = transform(self.X[index][0])
        return (img, label)

In [ ]:
training_data = np.load('gdrive/My Drive/Colab Notebooks/Dataset/training_data.npy', allow_pickle=True)

In [ ]:
training_data.shape

(24946, 2)

In [ ]:
dataset = CollectData(training_data)

In [ ]:
len(dataset)

24946

In [ ]:
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [19946, 5000])
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                           shuffle=False)

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3,64,3, padding='same')
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(64, 128, 3, padding='same')
        self.fc1 = nn.Linear(128*8*8, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 128*8*8)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (image, labels) in enumerate(train_loader):
        output = model(image)
        loss = criterion(output, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 1000 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')
    print(f"Epoch {epoch}: Loss: {loss.item():.4f} and only loss: {loss}")



Epoch 0: Loss: 0.6901 and only loss: 0.6901334524154663
Epoch 1: Loss: 0.6866 and only loss: 0.6865555047988892
Epoch 2: Loss: 0.6911 and only loss: 0.6910544633865356
Epoch 3: Loss: 0.6905 and only loss: 0.6905495524406433
Epoch 4: Loss: 0.6907 and only loss: 0.6906701326370239
Epoch 5: Loss: 0.6859 and only loss: 0.6858856081962585
Epoch 6: Loss: 0.6587 and only loss: 0.6587393283843994
Epoch 7: Loss: 0.6464 and only loss: 0.6464282870292664
Epoch 8: Loss: 0.6541 and only loss: 0.654095470905304
Epoch 9: Loss: 0.6651 and only loss: 0.6650972366333008
Epoch 10: Loss: 0.5819 and only loss: 0.5818589329719543
Epoch 11: Loss: 0.5889 and only loss: 0.5889167785644531
Epoch 12: Loss: 0.6596 and only loss: 0.6596115231513977
Epoch 13: Loss: 0.6007 and only loss: 0.600652813911438
Epoch 14: Loss: 0.6282 and only loss: 0.6282467842102051
Epoch 15: Loss: 0.5579 and only loss: 0.5579371452331543
Epoch 16: Loss: 0.5539 and only loss: 0.5538761615753174
Epoch 17: Loss: 0.6552 and only loss: 0.655

In [ ]:
for image, labels in test_loader:
    print(labels.shape)
    break

torch.Size([32])


In [ ]:
classes = ('Cat', 'Dog')

In [ ]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(2)]
    n_class_samples = [0 for i in range(2)]

    for image, labels in test_loader:
        #print(labels)
        output = model(image)
        _, predicted = torch.max(output, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1
    acc = 100.0 * n_correct / n_samples
    print(f"Accuracy of the network: {acc} %")

    for i in range(2):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f"Accuracy of {classes[i]}: {acc}%")

Accuracy of the network: 71.22 %
Accuracy of Cat: 59.2607472880675%
Accuracy of Dog: 83.07447232178416%


In [ ]:
import cv2 as cv
test_cat = cv.imread("dog.jpg")
test_cat = cv.resize(test_cat, (32, 32))
test_cat = transform(test_cat)
test_cat = test_cat.reshape(1,3,32,32)
output = model(test_cat)
predicted = torch.argmax(output)
predicted, output

(tensor(1), tensor([[-1.7785,  1.9813]], grad_fn=<AddmmBackward>))

In [ ]:
dataiter = iter(train_loader)
images, labels = dataiter.next()

In [ ]:
images.shape

torch.Size([25, 3, 32, 32])

In [ ]:
conv1 = nn.Conv2d(3,64,3, padding='same')
pool = nn.MaxPool2d(2,2)
conv2 = nn.Conv2d(64, 128, 3, padding='same')

In [ ]:
x = conv1(images)
print(x.shape)
x = pool(x)
print(x.shape)
x = conv2(x)
print(x.shape)
x = pool(x)
print(x.shape)

torch.Size([25, 64, 32, 32])
torch.Size([25, 64, 16, 16])
torch.Size([25, 128, 16, 16])
torch.Size([25, 128, 8, 8])
